In [3]:
import sys

project_dir = '/Users/alex/ml_class/project/'

sys.path.append(project_dir)

In [4]:
from core.rolling_analysis import RollingAnalyzer
from core.utils import get_dfs, get_good_dfs, \
get_close_prices, fix_gaps_in_data, calc_returns, \
calc_log_prices, get_pairs
from sklearn.preprocessing import FunctionTransformer

import warnings
import numpy
import pandas as pd



warnings.simplefilter(action='ignore', category=FutureWarning)
m = 100
start_date = '2019-01-01'
end_date = '2019-03-01'

print("getting data")
dfs = get_dfs(project_dir + "data")
print("filtering to good ones")
good_dfs = get_good_dfs(dfs, 30000)
print("getting close prices")
closes = get_close_prices(good_dfs)
print("filling gaps")
closes = fix_gaps_in_data(closes)
print("filtering to USDT")
pairs = get_pairs(good_dfs)
closes_usdt = closes[pairs['USD']]

print("calculating log prices")
log_df = calc_log_prices(closes_usdt)
filtered_df = log_df[start_date:end_date]

print("calculating returns")
returns = calc_returns(closes_usdt)
print("-------------------")
print("Now rolling analysis")
rolling_analyzer = RollingAnalyzer(filtered_df, m)
print("Make the dfs")
rolling_analyzer.make_dfs()
rolling_analyzer.make_pairs()
print("Running all the regressions")
rolling_analyzer.run_all_regressions()
rolling_analyzer.calc_all_returns_stats()

print("Calculating dickey fullers")
rolling_analyzer.run_regressions('ETHUSDT', 'BTCUSDT')
rolling_analyzer.calc_dickey_fuller('ETHUSDT', 'BTCUSDT')

getting data
filtering to good ones
getting close prices
filling gaps
filtering to USDT
calculating log prices
calculating returns
-------------------
Now rolling analysis
Make the dfs
Running all the regressions
('LTCUSDT', 'BNBUSDT')
('TRXUSDT', 'BNBUSDT')
('TRXUSDT', 'LTCUSDT')
('ETHUSDT', 'BNBUSDT')
('ETHUSDT', 'LTCUSDT')
('ETHUSDT', 'TRXUSDT')
('BTCUSDT', 'BNBUSDT')
('BTCUSDT', 'LTCUSDT')
('BTCUSDT', 'TRXUSDT')
('BTCUSDT', 'ETHUSDT')
('EOSUSDT', 'BNBUSDT')
('EOSUSDT', 'LTCUSDT')
('EOSUSDT', 'TRXUSDT')
('EOSUSDT', 'ETHUSDT')
('EOSUSDT', 'BTCUSDT')
('ADAUSDT', 'BNBUSDT')
('ADAUSDT', 'LTCUSDT')
('ADAUSDT', 'TRXUSDT')
('ADAUSDT', 'ETHUSDT')
('ADAUSDT', 'BTCUSDT')
('ADAUSDT', 'EOSUSDT')
('XRPUSDT', 'BNBUSDT')
('XRPUSDT', 'LTCUSDT')
('XRPUSDT', 'TRXUSDT')
('XRPUSDT', 'ETHUSDT')
('XRPUSDT', 'BTCUSDT')
('XRPUSDT', 'EOSUSDT')
('XRPUSDT', 'ADAUSDT')
Calculating dickey fullers


# Feature Engineering

Now that all the rolling calculations have been executed, we can export and further manipulate them for model input. To run the regression on previous values (somewhat like an ARIMA model), I lag all the data. I also put together the $Y$ matrix which is what will ultimately be predicted.

In [13]:
import numpy as np
import matplotlib.pyplot as plt

def rename_columns(df, tag):
    new_column_names = [col + '_' + tag for col in df.columns]
    df.columns = new_column_names
    return df

mean_returns_df = rename_columns(rolling_analyzer.export_returns_stats_df(stat='mean_return'), 'mean')
volatility_df = rename_columns(rolling_analyzer.export_returns_stats_df(stat='volatility'), 'vol')
dickey_fuller_df = rename_columns(rolling_analyzer.export_dickey_fuller_df(), 'pval')
betas_df = rename_columns(rolling_analyzer.export_betas_df(), 'betas')

In [14]:
def zero_to_one(x):
    return np.exp(x)/(1+np.exp(x))

def to_infinity(x):
    return np.log(x/(1-x))

inf_dickey_fuller = to_infinity(dickey_fuller_df)

/Users/alex/ml_class/project_venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


In [15]:
def calculate_lags(features, target, m):
    target_lags = {}
    for i in range(0, m):
        code = 'y' + str(i)
        lag = target.shift(-i).iloc[:,0]
        target_lags[code] = lag

    y = pd.concat(target_lags, axis=1)

    features_df = pd.concat(features, axis=1).dropna()
    feature_lags = []
    for i in range(0, m):
        lag_df = features_df.shift(i)
        lag_df.columns = [c + '_lag' + str(i) for c in list(features_df.columns)]
        feature_lags.append(lag_df)

    feature_lags = pd.concat(feature_lags, axis=1)
    full_df = pd.concat([feature_lags, y], axis=1).dropna()
    full_df = full_df.replace([np.inf, -np.inf], 0)
    out = {
        'full_df': full_df,
        'features': feature_lags,
        'target': y
    }
    return out

lagged_data = calculate_lags([mean_returns_df, volatility_df, inf_dickey_fuller], inf_dickey_fuller, 100)

In [16]:
def make_train_test(df, ratio = 0.2, target=['y']):
    split_point = int((1 - ratio) * df.shape[0])
    y = df[target]
    x = df.drop(target, axis=1)
    y_train = y[0:split_point]
    x_train = x[0:split_point]
    y_test = y[(split_point + 1):]
    x_test = x[(split_point + 1):]
    out = {'train': {'x': x_train, 'y': y_train},
           'test': {'x': x_test, 'y': y_test}
          }
    return out
    
full_set = make_train_test(lagged_data['full_df'], target=list(lagged_data['target'].keys()))

# Dimensionality Reduction

The dataset has an incredibly large number of features as well as 100 output target variables. I elect to use PCA to reduce *both* of these, the input $X$ matrix as well as the output $Y$ matrix.

In [152]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def do_pca_reduce(dataset):
    pca_pipeline = Pipeline([
      ('scale', StandardScaler()),
      ('pca', PCA(n_components=0.9))
    ])

    x_train_pca_reduced = pd.DataFrame(pca_pipeline.fit_transform(dataset['train']['x']))
    y_train_pca_reduced = pd.DataFrame(pca_pipeline.fit_transform(dataset['train']['y']))
    x_test_pca_reduced = pd.DataFrame(pca_pipeline.fit_transform(dataset['test']['x']))
    y_test_pca_reduced = pd.DataFrame(pca_pipeline.fit_transform(dataset['test']['y']))
  
    x_train_pca_reduced.index = dataset['train']['x'].index
    y_train_pca_reduced.index = dataset['train']['y'].index
    x_test_pca_reduced.index = dataset['test']['x'].index
    y_test_pca_reduced.index = dataset['test']['y'].index

    out = {
    'pipeline': pca_pipeline,
    'original_data': dataset,
    'pca_data': {
            'train': {'x': x_train_pca_reduced, 'y': y_train_pca_reduced},
            'test': {'x': x_test_pca_reduced, 'x': x_test_pca_reduced}
        }
    }
    return out

In [154]:
def rebuild_df(pca_df):
    rebuilt_df = pd.DataFrame(pca_pipeline.inverse_transform(pca_df))
    rebuilt_df.index = pca_df.index
    rebuilt_df.columns = ['y' + str(i) for i in range(rebuilt_df.shape[1])]
    return rebuilt_df

In [155]:
pca_data = do_pca_reduce(full_set)

In [156]:
full_set['train']['x'].shape, full_set['train']['y'].shape

((4369, 1700), (4369, 100))

In [157]:
pca_data['pca_data']['train']['x'].shape, pca_data['pca_data']['train']['y'].shape

((4369, 20), (4369, 15))

# Support Vector Regression

My model uses support vector machines to forecast the next hundred observations. The `MultiOutputRegressor` class is used to implement the multivariate output. I do a randomized search for hyperparameter tuning.

In [158]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor

scaler = StandardScaler()

x = pca_data['pca_data']['train']['x']
y = pca_data['pca_data']['train']['y']

x_scaled = pd.DataFrame(scaler.fit_transform(x))
x_scaled.index = x.index

multioutput_model = MultiOutputRegressor(estimator=SVR())

In [179]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit

tseries_split = TimeSeriesSplit(n_splits=5)
spl = tseries_split.split(x_scaled)

In [180]:
params = {
    "estimator__C": [0, 0.25, 0.5, 1, 2, 5, 10],
    "estimator__kernel": ['sigmoid', 'poly', 'linear']
}

randomized_search = RandomizedSearchCV(
    multioutput_model,
    param_distributions=params,
    cv=spl
)

In [181]:
randomized_search.fit(x_scaled, y)

ValueError: C <= 0

In [162]:
randomized_search.best_params_

{'estimator__kernel': 'poly', 'estimator__C': 0.5}

In [133]:
model = randomized_search.best_estimator_

In [165]:
predicted = pd.DataFrame(
    model.predict(x_scaled),
    index=full_set['train']['x'].index
)

In [168]:
def rolled_mse(true_values, predicted, transform = True):
    mse_list = []
    for i in range(true_values.shape[0]):
        temp_true = true_values.iloc[i, :]
        temp_pred = predicted.iloc[i, :]
        if transform:
            temp_true = zero_to_one(temp_true)
            temp_pred = zero_to_one(temp_pred)
        temp_mse = mean_squared_error(temp_true, temp_pred)
        mse_list.append(temp_mse)
    return mse_list

def overall_rolled_mse(true_values, predicted, transform=True):
    mse_list = rolled_mse(true_values, predicted, transform)
    return np.mean(mse_list)

full_predicted = rebuild_df(predicted)

mse_list = rolled_mse(
    full_set['train']['y'],
    full_predicted
)

np.sqrt(overall_rolled_mse(full_set['train']['y'], full_predicted))

NameError: name 'pca_pipeline' is not defined

In [ ]:
def time_series_cross_val_split(x, y, predict_n=100):
    n = x.shape[0]
    split = n - predict_n
    in_sample_x = x.iloc[:(split - 1), :]
    in_sample_y = y.iloc[:(split - 1)]
    out_of_sample_x = x.iloc[split:, :]
    out_of_sample_y = y.iloc[split:, :]
    out = {
        'in': {'x': in_sample_x, 'y': in_sample_y},
        'out': {'x': out_of_sample_x, 'y': out_of_sample_y}
    }
    return out
    
split = time_series_cross_val_split(pd.DataFrame(x_scaled), y_pca_reduced)

In [ ]:
def time_series_cross_val_predict(model, x, y, folds=10):
    if folds < 5:
        print("Must have 5 or more folds")
        return
    else:
        x_folds = time_series_kfolds(x, folds=folds)
        y_folds = time_series_kfolds(y, folds=folds)
        predictions = []
        true_values = []
        for i in range(3, folds):
            x_train = pd.concat([x_folds[i-3], x_folds[i-2], x_folds[i-1]], axis=0)
            y_train = pd.concat([y_folds[i-3], y_folds[i-2], y_folds[i-1]], axis=0)
            x_test = x_folds[i]
            true_values.append(y_folds[i])
            model.fit(x_train, y_train)
            temp_predictions = model.predict(x_test)
            predictions.append(temp_predictions)
        out = {'true_values': true_values, 'predictions': predictions}
        return out

In [ ]:
pipeline = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('svr', SVR(C=0.5, kernel='poly'))
])

cross_val_predictions = time_series_cross_val_predict(
    pipeline,
    full_set['train']['x'],
    full_set['train']['y'], folds=20
)